In [1]:
import pandas as pd
import numpy as np
import matplotlib as plot 
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
file_path = r'C:\Users\Lorenzo\Desktop\DSTI\DSTI Courses\ML Project\DSTI_ML-Project\books.csv'
df = pd.read_csv(file_path, on_bad_lines='skip')
df = df.drop('isbn', axis=1)
df = df.drop('isbn13', axis=1)
df.head()

,bookID,title,authors,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   language_code       11123 non-null  object 
 5     num_pages         11123 non-null  int64  
 6   ratings_count       11123 non-null  int64  
 7   text_reviews_count  11123 non-null  int64  
 8   publication_date    11123 non-null  object 
 9   publisher           11123 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 869.1+ KB


In [ ]:
df['publication_year'] = df['publication_date'].str[-4:].astype(int)
df = df.drop('publication_date', axis=1)

In [ ]:
df.head(15)

,bookID,title,authors,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publisher,publication_year
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,Scholastic Inc.,2006
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,Scholastic Inc.,2004
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,Scholastic,2003
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,Scholastic Inc.,2004
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,Scholastic,2004
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,en-US,152,19,1,Nimble Books,2005
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,eng,3342,28242,808,Scholastic,2005
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,eng,815,3628,254,Gramercy Books,2005
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,eng,815,249558,4080,Del Rey Books,2002
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,eng,215,4930,460,Crown,2004


In [ ]:
unique_publishers = df['publisher'].unique()
 
df_uniquepublisher = pd.DataFrame(unique_publishers, columns=['publisher'])
df_uniquepublisher[['publisher1', 'publisher2', 'publisher3','publisher4','publisher5','publisher6','publisher7']] = df_uniquepublisher['publisher'].str.split(" ", n=6, expand=True)

df_uniquepublisher.head()

,publisher,publisher1,publisher2,publisher3,publisher4,publisher5,publisher6,publisher7
0,Scholastic Inc.,Scholastic,Inc.,None,None,None,None,None
1,Scholastic,Scholastic,None,None,None,None,None,None
2,Nimble Books,Nimble,Books,None,None,None,None,None
3,Gramercy Books,Gramercy,Books,None,None,None,None,None
4,Del Rey Books,Del,Rey,Books,None,None,None,None


In [ ]:
df['publisherID'] = df['publisher'].astype('category').cat.codes
df['authorsID'] = df['authors'].str.split("/").str[0].astype('category').cat.codes
df['contributor1ID'] = df['authors'].str.split("/").str[1]
df['contributor1ID'] = np.where(df['contributor1ID'].isna(), np.nan, df['contributor1ID'].astype('category').cat.codes)
df['contributor1ID'] = pd.to_numeric(df['contributor1ID'], errors='coerce').astype('Int64')
df['Laguage_codeID'] = df['language_code'].astype('category').cat.codes

df.to_csv('dataframe.csv')

df.head()


,bookID,title,authors,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publisher,publication_year,publisherID,authorsID,contributor1ID,Laguage_codeID
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,Scholastic Inc.,2006,1778,1598,2031,5
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,Scholastic Inc.,2004,1778,1598,2031,5
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,Scholastic,2003,1773,1598,<NA>,5
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,Scholastic Inc.,2004,1778,1598,2031,5
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,Scholastic,2004,1773,1598,2031,5


In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def embed_text(text):
    return embed([text]).numpy()[0]

df['title_embedding'] = df['title'].apply(embed_text)
df.head()

,bookID,title,authors,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publisher,publication_year,...,番外編,若獅子戦,野獸國,鋼の錬金術師,鋼之鍊金術師,院生試験,雙城奇謀,魔戒二部曲,魔戒現身,魔戒首部曲
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,Scholastic Inc.,2006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,Scholastic Inc.,2004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,Scholastic,2003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,Scholastic Inc.,2004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,Scholastic,2004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_ML = df.drop(['title', 'authors', 'publisher','language_code'], axis = 1)
corr_matrix = df_ML.corr()
corr_matrix_np = corr_matrix.values

print(corr_matrix_np)

# To-do List
1. Find a way to use the title (convert it to float or int)
2. Start to analyze the data patterns (correlation, plotting)
3. Choose a model ? Regression ? Classification ?

For the 19/07 Start to work on the ML Model